## ESG controversy analysis - Modeling

In [1]:
# Import packages
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
#from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_class_weight
from lightgbm import LGBMClassifier
import re

from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, auc
from sklearn.ensemble import AdaBoostClassifier

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import plotly.express as px
import plotly.graph_objs as go 
from plotly.graph_objects import Layout
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import ADASYN

from sklearn.metrics import confusion_matrix

In [70]:
import plotly.io as pio

# Define Custom Theme
pio.templates['master_thesis'] = go.layout.Template(
    layout=go.Layout(
        font = dict(family= 'Times New Roman', size = 30),
        xaxis = dict(zeroline = True, 
                    linewidth = 1, 
                    linecolor = 'black', 
                    title_font=dict(size=35),
                    mirror=False,
                    showline=True,
                    gridcolor='white'),
        yaxis = dict(zeroline = False,
                    rangemode = 'tozero', 
                    linewidth = 1, 
                    linecolor = 'black', 
                    title_font=dict(size=35), 
                    mirror=False,
                    showline=True,
                    gridcolor='white'),
        colorway=['#0055B3', '#FF2400', '#028A0F'],
        margin=dict(l=100, r=0, t=0, b=100),
        legend=dict(yanchor="top",
            y=0.98,
            xanchor="left",
            x=1.03,
            title = None,
            font=dict(size= 20),
            bordercolor="Black",
            borderwidth=1),
        plot_bgcolor='rgb(242, 242, 242)',
        ),
)
pio.templates.default = 'master_thesis'

### Modeling

In [3]:
# Import data
os.chdir(
    r"//Users/mlvos/Desktop/Moritz/Education/Erasmus University/Master/Master Thesis_code/"
)

df_merged = pd.read_csv("data/merged_data.csv", index_col=['id', 'year'])

In [4]:
df_merged.shape

(29979, 109)

In [5]:
# Define the columns to be one-hot encoded
categorical_cols = df_merged.select_dtypes(include=['object']).columns.tolist()
categorical_cols = categorical_cols[1:]

In [6]:
# Create empty dataframe that compares output

df_results = pd.read_csv("data/results_environmental.csv")
# df_results = pd.DataFrame(columns=[
#                           'Model', 'Parameters', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC', 'Best params'])

**Logistic Regression (perhaps with imputation and regularisation)**

In [7]:
# Split the data into training and testing sets
#df_merged.dropna(axis=0, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(df_merged.drop(['ISIN Code', 'ESG Controversies Score', 'ESG Combined Score', 'GICS Industry Group Name', 'country',
                                                                    'Environmental Controversies Count','Social Controversies Count',
                                                                    'Governance Controversies Count',
                                                                    'Governance_controversy_binary',
                                                                    'Social_controversy_binary',
                                                                    'Environmental_controversy_binary',
                                                                    'Recent Governance Controversies',
                                                                    'Recent Social Controversies',
                                                                    'Governance_controversy_binary'], axis=1), 
                                                    df_merged['Environmental_controversy_binary'], stratify=df_merged['Environmental_controversy_binary'], test_size=0.3, random_state=42)

In [8]:
# Compute class weights for logistic regression
class_weights = compute_class_weight('balanced', classes=[0, 1], y=y_train)

# One-hot encode vairables
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
#     ]
# )

# Create a pipeline with two steps: StandardScaler and LogisticRegression
pipe = Pipeline([#('preprocessor', preprocessor), 
                 ('imputer', KNNImputer(metric='nan_euclidean')),
                 ('smote', SMOTE(random_state=42, sampling_strategy='minority')),
                 #('ada', ADASYN()),
                 ('lr', LogisticRegression(random_state=42, max_iter=5000, solver='saga', class_weight={0: class_weights[0], 1: class_weights[1]}))]) 
# Define a param_grid for GridSearchCV that includes the regularization parameter C
param_grid = {
    'imputer__n_neighbors': [7], 
    'lr__C': [10], 
    'lr__penalty': ['l1']
}              

In [9]:
print(class_weights[0], class_weights[1])

0.5064681179707486 39.151119402985074


In [10]:
# Fit the pipeline with GridSearchCV to the training data
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=cv, verbose = 1, n_jobs=6, scoring = 'f1')
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/mlvos/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mlvos/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mlvos/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mlvos/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mlvos/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mlvos/opt/anaconda3/lib/python3.9/site-pack

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('imputer', KNNImputer()),
                                       ('smote',
                                        SMOTE(random_state=42,
                                              sampling_strategy='minority')),
                                       ('lr',
                                        LogisticRegression(class_weight={0: 0.5064681179707486,
                                                                         1: 39.151119402985074},
                                                           max_iter=5000,
                                                           random_state=42,
                                                           solver='saga'))]),
             n_jobs=6,
             param_grid={'imputer__n_neighbors': [7], 'lr__C': [10],
                         'lr__penalty': ['l1']},
             scoring='f1', verbose=1)

In [11]:
# Use the best estimator from GridSearchCV to predict on the testing data
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [12]:
# Check paramters of best performing model
best_params

{'imputer__n_neighbors': 7, 'lr__C': 10, 'lr__penalty': 'l1'}

In [13]:
# Predict on y test
y_pred = best_model.predict(X_test)

# Evaluate the model performance
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred))

Accuracy: 0.09195018901489882
Precision: 0.013650640251268423
Recall: 0.9826086956521739
F1 score: 0.026927201239127847


In [14]:
# Append to dataframe
df_results = df_results.append({'Model': 'LR_smote_final', 'Accuracy': accuracy_score(y_test, y_pred),
                                'Parameters': pipe.named_steps,
                                'Precision': precision_score(y_test, y_pred),
                                'Recall': recall_score(y_test, y_pred),
                                'F1 Score': f1_score(y_test, y_pred),
                                'AUC': roc_auc_score(y_test, y_pred),
                                'Best params': best_params},
                               ignore_index=True)

#print(df_results)

# write to csv
df_results.to_csv(r"data/results_environmental.csv")

/var/folders/kp/_bs5hp_966j6kxznvkczsqdh0000gn/T/ipykernel_8787/1806535987.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Model': 'LR_smote_final', 'Accuracy': accuracy_score(y_test, y_pred),


In [15]:
# Predict on y test
y_pred = best_model.predict_proba(X_test)[::,1]

fpr, tpr, _ = roc_curve(y_test,  y_pred)

fpr_lr, tpr_lr = fpr, tpr

# Compute AUC (Area Under the Curve)
auc_ = roc_auc_score(y_test, y_pred)

auc_lr = auc_

# first attempt
fig_log_auc = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC={auc_:.2f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig_log_auc.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig_log_auc.update_yaxes(scaleanchor="x", scaleratio=1)
fig_log_auc.update_xaxes(constrain='domain')
fig_log_auc.show()

fig_log_auc.write_image("images/results/ROC_environemntal_lr.png")

In [16]:
# Get number of positive classes and rows in data set
positives = list(df_merged[df_merged['Environmental_controversy_binary'] > 0].shape)[0]
rows = list(df_merged.shape)[0]
baseline = positives/rows

In [17]:
precision, recall, thresholds = precision_recall_curve(y_test,  y_pred)
auc_precision_recall = auc(recall, precision)

In [18]:
# Predict on y test
y_pred = best_model.predict_proba(X_test)[::,1]

precision, recall, thresholds = precision_recall_curve(y_test,  y_pred)

precision_lr, recall_lr = precision, recall

auc_precision_recall = auc(recall, precision)

auc_precision_recall_lr = auc_precision_recall

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve (AUC={auc_precision_recall:.2f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=baseline, y1=baseline
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.add_annotation(text=f'Baseline: {baseline:.2f}',
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=1.0,
                    y=baseline,
                    bordercolor='black',
                    borderwidth=1)

fig.show()

fig.write_image("images/results/PRC_environmental_lr.png")

In [19]:
fig = go.Figure(layout=go.Layout(width=700, height=500
    ))

#title=go.layout.Title(text="Precision and Recall by Threshold")

fig.add_trace(go.Scatter(name="Precision", x=thresholds, y=precision[:-1]))
fig.add_trace(go.Scatter(name="Recall", x=thresholds, y=recall[:-1]))
fig.update_xaxes(title="Threshold")
fig.update_yaxes(title="Proportion")
fig

fig.write_image("images/results/PCR_threshold_environmental_lr.png")

In [20]:
# Generate a range of threshold values
thresholds = np.arange(0.01, 1.00, 0.01)

# Loop through threshold values and calculate F1 score for each
f1_scores = []
for threshold in thresholds:
    y_pred_2 = (y_pred >= threshold).astype(int)
    f1_scores.append(f1_score(y_test, y_pred_2))

# Find optimal threshold value that maximizes F1 score
optimal_threshold = thresholds[np.argmax(f1_scores)]

# Evaluate the model performance
y_pred_2 = (y_pred >= optimal_threshold).astype(int)
print('Accuracy:', accuracy_score(y_test, y_pred_2))
print('Precision:', precision_score(y_test, y_pred_2))
print('Recall:', recall_score(y_test, y_pred_2))
print('F1 score:', f1_score(y_test, y_pred_2))

Accuracy: 0.2021347565043362
Precision: 0.014574453457995325
Recall: 0.9217391304347826
F1 score: 0.028695181375203032


In [21]:
# Print optimal threshold
print('optimal threshold:', optimal_threshold)

optimal threshold: 0.98


In [22]:
# Display confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_2).ravel()
print('tn:', tn, 'fp:', fp, 'fn:', fn, 'tp:', tp)

tn: 1712 fp: 7167 fn: 9 tp: 106


**Light Gradient Boosting**

In [23]:
df_merged_gbm = df_merged.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df_merged_gbm.dropna(inplace=True)

In [24]:
# Define the columns to be one-hot encoded
categorical_cols_gbm = df_merged_gbm.select_dtypes(include=['object']).columns.tolist()
categorical_cols_gbm = categorical_cols[:]

In [25]:
X_train_gbm, X_test_gbm, y_train_gbm, y_test_gbm = train_test_split(df_merged_gbm.drop(['ISINCode', 'ESGCombinedScore', 'ESGControversiesScore', 'GICSIndustryGroupName', 'country',
                                                                    'EnvironmentalControversiesCount','SocialControversiesCount',
                                                                    'GovernanceControversiesCount',
                                                                    'Governance_controversy_binary',
                                                                    'Environmental_controversy_binary',
                                                                    'Governance_controversy_binary',
                                                                    'RecentGovernanceControversies',
                                                                    'RecentSocialControversies',
                                                                    'Social_controversy_binary'], axis=1), 
                                                    df_merged_gbm['Environmental_controversy_binary'], stratify=df_merged_gbm['Environmental_controversy_binary'], test_size=0.3, random_state=42)

In [26]:
#('r', SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))), 
# Compute class weights for logistic regression
class_weights = compute_class_weight('balanced', classes=[0, 1], y=y_train_gbm)

# # One-hot encode vairables
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols_gbm)
#     ]
# )


# Create a pipeline with two steps: StandardScaler and LogisticRegression
pipe = Pipeline([#('scaler', StandardScaler()),
                ('imputer', KNNImputer(metric='nan_euclidean')),
                ('smote', SMOTE(random_state=42, sampling_strategy='minority')),
                #('preprocessor', preprocessor), # without preprocessing much higher dont know why!!
                #('resample', TomekLinks(sampling_strategy='majority')), #SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))),
                #('ada', ADASYN()), #gives quite a balanced result - p .25, recall .26
                ('classifier', LGBMClassifier(random_state=42, class_weight={0: class_weights[0], 1: class_weights[1]}))]) #class_weight={0: class_weights[0], 1: class_weights[1]}

# Define a param_grid for GridSearchCV that includes the regularization parameter C
param_grid = {
    'classifier__learning_rate': [0.1],
    'classifier__max_depth': [5],
    'classifier__n_estimators': [100],
    'imputer__n_neighbors': [3],
    'classifier__boosting_type': ['gbdt'],
    'classifier__num_leaves': [31]
}

In [27]:
# Fit the pipeline with GridSearchCV to the training data
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=cv, verbose = 1, n_jobs=6, scoring = 'f1')
grid_search.fit(X_train_gbm, y_train_gbm)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('imputer', KNNImputer()),
                                       ('smote',
                                        SMOTE(random_state=42,
                                              sampling_strategy='minority')),
                                       ('classifier',
                                        LGBMClassifier(class_weight={0: 0.5079260237780714,
                                                                     1: 32.041666666666664},
                                                       random_state=42))]),
             n_jobs=6,
             param_grid={'classifier__boosting_type': ['gbdt'],
                         'classifier__learning_rate': [0.1],
                         'classifier__max_depth': [5],
                         'classifier__n_estimators': [100],
                         'classifier__num_leaves': [31],
                         'imputer

In [28]:
# Use the best estimator from GridSearchCV to predict on the testing data
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [29]:
best_params

{'classifier__boosting_type': 'gbdt',
 'classifier__learning_rate': 0.1,
 'classifier__max_depth': 5,
 'classifier__n_estimators': 100,
 'classifier__num_leaves': 31,
 'imputer__n_neighbors': 3}

In [30]:
# Predict on y test
y_pred_gbm = best_model.predict(X_test_gbm)

# Evaluate the model performance
print('Accuracy:', accuracy_score(y_test_gbm, y_pred_gbm))
print('Precision:', precision_score(y_test_gbm, y_pred_gbm))
print('Recall:', recall_score(y_test_gbm, y_pred_gbm))
print('F1 score:', f1_score(y_test_gbm, y_pred_gbm))

Accuracy: 0.9683571738188123
Precision: 0.1864406779661017
Recall: 0.3055555555555556
F1 score: 0.23157894736842105


In [31]:
# Append to dataframe
df_results = df_results.append({'Model': 'GBM_smote_final', 'Accuracy': accuracy_score(y_test_gbm, y_pred_gbm),
                                'Parameters': pipe.named_steps,
                                'Precision': precision_score(y_test_gbm, y_pred_gbm),
                                'Recall': recall_score(y_test_gbm, y_pred_gbm),
                                'F1 Score': f1_score(y_test_gbm, y_pred_gbm),
                                'AUC': roc_auc_score(y_test_gbm, y_pred_gbm),
                                'Best params': best_params},
                               ignore_index=True)

#print(df_results)

# write to csv
df_results.to_csv(r"data/results_environmental.csv")

/var/folders/kp/_bs5hp_966j6kxznvkczsqdh0000gn/T/ipykernel_8787/1250697825.py:2: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [32]:
# Predict on y test
y_pred_gbm = best_model.predict_proba(X_test_gbm)[::,1]

# Assuming y_pred and y_true are the predicted and true labels, respectively
fpr, tpr, _ = roc_curve(y_test_gbm, y_pred_gbm)

fpr_lgbm, tpr_lgbm = fpr, tpr

# Compute AUC (Area Under the Curve)
auc_ = roc_auc_score(y_test_gbm, y_pred_gbm)

auc_lgbm = auc_

# first attempt
fig_rf_auc = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC={auc_:.2f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig_rf_auc.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig_rf_auc.update_yaxes(scaleanchor="x", scaleratio=1)
fig_rf_auc.update_xaxes(constrain='domain')
fig_rf_auc.show()

fig_rf_auc.write_image("images/results/ROC_environemntal_lgbm.png")

In [33]:
# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Train score: ", grid_search.best_score_)
print("Test score: ", grid_search.score(X_test_gbm, y_test_gbm))

Best parameters:  {'classifier__boosting_type': 'gbdt', 'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__n_estimators': 100, 'classifier__num_leaves': 31, 'imputer__n_neighbors': 3}
Train score:  0.16299501170721778
Test score:  0.23157894736842105


In [34]:
# Predict on y test
y_pred = best_model.predict_proba(X_test_gbm)[::,1]

precision, recall, thresholds = precision_recall_curve(y_test_gbm,  y_pred_gbm)

precision_lgbm, recall_lgbm = precision, recall

auc_precision_recall = auc(recall, precision)

auc_precision_recall_lgbm = auc_precision_recall

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve (AUC={auc_precision_recall:.2f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=baseline, y1=baseline
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.add_annotation(text=f'Baseline:{baseline:.4f}',
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=1.0,
                    y=baseline,
                    bordercolor='black',
                    borderwidth=1)

fig.show()

fig.write_image("images/results/PRC_environmental_lgbm.png")

In [37]:
fig = go.Figure(layout=go.Layout(width=700, height=500 #title=go.layout.Title(text="Precision and Recall by Threshold")
    ))

fig.add_trace(go.Scatter(name="Precision", x=thresholds, y=precision[:-1]))
fig.add_trace(go.Scatter(name="Recall", x=thresholds, y=recall[:-1]))
fig.update_xaxes(title="Threshold")
fig.update_yaxes(title="Proportion")

fig.show()

fig.write_image("images/final/environmental/PCR_threshold_environmental_lgbm.png")

pio.write_image(fig, 'images/final/environmental/PCR_threshold_environmental_lgbm.png', scale=5, width=700, height=500)

In [38]:
from sklearn.inspection import permutation_importance

# Calculate permutation feature importance
result = permutation_importance(
    grid_search, X_test_gbm, y_test_gbm, n_repeats=10, random_state=42, n_jobs=6
)

feature_names = X_train_gbm.columns

# Put the result in a dataframe
forest_importances = pd.DataFrame(result.importances_mean, columns=['importance'], index=feature_names)

# Add standard deviations to the dataframe
forest_importances['std'] = result.importances_std

# Sort the dataframe by largest values
forest_importances.sort_values(by='importance', ascending=False, inplace=True)

In [39]:
# Store top 20 in dataframe
feature_import = pd.DataFrame(forest_importances['importance'][:20])
feature_import.index = ['Total Liabilites', 
                        'Health Safety Policy Score',
                        'Susatinability Compensation Incentives Score',
                                           'Board Functions Policy Score',
                                           'Announced Layoofs to Total Employees Score',
                                           'Nomination Committee Score',
                                           'Resource Reduction Policy Score',
                                           'Policy Fair Competition Score',
                                           'Internal Audit Department Reporting Score',
                                           'Executive Compensation Objectives Score',
                                           'Executive Members Gender Diversity Score',
                                           'Board Individual Reelection Score',
                                           'Board Structure Policy Score',
                                           'Executive Compensation Policy Score',
                                           'Environmental Innovation Score',
                                           'Employees Health Safety Team Score',
                                           'Board Member Affiliations Score',
                                           'Shareholders Approval Stock Compensation Score',
                                           'Board Background and Skills Score',
                                           'Policy Environmental Supply Chain Score']

In [52]:
fig = px.bar(feature_import, y="importance", width=800, 
                           height=900) #, title="Feature importances using permutation on full model"
fig.update_yaxes(title = 'Importance')
fig.update_xaxes(title = None)

fig.update_layout(margin=dict(l=120, r=20, t=20, b=600),)

fig.show()

fig.write_image("images/final/environmental/feature_importance_environmental_lgbm.png")
pio.write_image(fig, 'images/final/environmental/feature_importance_environmental_lgbm.png', scale=5, width=800, height=900)

In [53]:
# Generate a range of threshold values
thresholds = np.arange(0.01, 1.00, 0.01)

# Loop through threshold values and calculate F1 score for each
f1_scores = []
for threshold in thresholds:
    y_pred_2 = (y_pred >= threshold).astype(int)
    f1_scores.append(f1_score(y_test_gbm, y_pred_2))

# Find optimal threshold value that maximizes F1 score
optimal_threshold = thresholds[np.argmax(f1_scores)]

# Evaluate the model performance
y_pred_2 = (y_pred >= optimal_threshold).astype(int)
print('Accuracy:', accuracy_score(y_test_gbm, y_pred_2))
print('Precision:', precision_score(y_test_gbm, y_pred_2))
print('Recall:', recall_score(y_test_gbm, y_pred_2))
print('F1 score:', f1_score(y_test_gbm, y_pred_2))

Accuracy: 0.9783268313827481
Precision: 0.28125
Recall: 0.25
F1 score: 0.2647058823529412


In [54]:
# Print optimal threshold
print('optimal threshold:', optimal_threshold)

optimal threshold: 0.8300000000000001


In [55]:
# Display confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test_gbm, y_pred_2).ravel()
print('tn:', tn, 'fp:', fp, 'fn:', fn, 'tp:', tp)

tn: 2248 fp: 23 fn: 27 tp: 9


**Quadratic Discriminant Analysis**

In [56]:
# # One-hot encode vairables
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
#     ]
# )

pipeline = Pipeline([#('preprocessor', preprocessor), 
                     ('imputer', KNNImputer(metric='nan_euclidean')),
                     #('ada', ADASYN()),
                     ('smote', SMOTE(random_state=42, sampling_strategy='minority')),
                     ('qda', QuadraticDiscriminantAnalysis())])

param_grid = {
    'imputer__n_neighbors': [5],
    'qda__reg_param': [0.2043359717856948] #list(np.logspace(-1.0, 1.0, 5))
}


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=cv, n_jobs=6, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('imputer', KNNImputer()),
                                       ('smote',
                                        SMOTE(random_state=42,
                                              sampling_strategy='minority')),
                                       ('qda',
                                        QuadraticDiscriminantAnalysis())]),
             n_jobs=6,
             param_grid={'imputer__n_neighbors': [5],
                         'qda__reg_param': [0.2043359717856948]},
             verbose=1)

In [57]:
# Use the best estimator from GridSearchCV to predict on the testing data
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [58]:
best_params

{'imputer__n_neighbors': 5, 'qda__reg_param': 0.2043359717856948}

In [59]:
# Predict on y test
y_pred = best_model.predict(X_test)

# Evaluate the model performance
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred))

Accuracy: 0.9808761396486546
Precision: 0.2
Recall: 0.16521739130434782
F1 score: 0.18095238095238095


In [60]:
# Append to dataframe
df_results = df_results.append({'Model': 'QDA_smote_final', 'Accuracy': accuracy_score(y_test, y_pred),
                                'Parameters': pipeline.named_steps,
                                'Precision': precision_score(y_test, y_pred),
                                'Recall': recall_score(y_test, y_pred),
                                'F1 Score': f1_score(y_test, y_pred),
                                'AUC': roc_auc_score(y_test, y_pred),
                                'Best params': best_params},
                               ignore_index=True)

#print(df_results)

# write to csv
df_results.to_csv(r"data/results_environmental.csv")

/var/folders/kp/_bs5hp_966j6kxznvkczsqdh0000gn/T/ipykernel_8787/3264458623.py:2: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [61]:
# Predict on y test
y_pred = best_model.predict_proba(X_test)[::,1]

fpr, tpr, _ = roc_curve(y_test,  y_pred)

fpr_qda, tpr_qda = fpr, tpr

# Compute AUC (Area Under the Curve)
auc_ = roc_auc_score(y_test, y_pred)

auc_qda = auc_

fig_qdc_auc = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC={auc_:.2f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig_qdc_auc.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig_qdc_auc.update_yaxes(scaleanchor="x", scaleratio=1)
fig_qdc_auc.update_xaxes(constrain='domain')
fig_qdc_auc.show()

fig_qdc_auc.write_image("images/results/ROC_environemntal_qda.png")

In [62]:
# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Train score: ", grid_search.best_score_)
print("Test score: ", grid_search.score(X_test, y_test))

Best parameters:  {'imputer__n_neighbors': 5, 'qda__reg_param': 0.2043359717856948}
Train score:  0.9829401953776508
Test score:  0.9808761396486546


In [63]:
# Predict on y test
y_pred = best_model.predict_proba(X_test)[::,1]

precision, recall, thresholds = precision_recall_curve(y_test,  y_pred)

precision_qda, recall_qda = precision, recall

auc_precision_recall = auc(recall, precision)

auc_precision_recall_qda = auc_precision_recall

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve (AUC={auc_precision_recall:.2f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=baseline, y1=baseline
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.add_annotation(text=f'Baseline: {baseline:.4f}',
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=1.0,
                    y=baseline,
                    bordercolor='black',
                    borderwidth=1)

fig.show()

fig.write_image("images/results/PRC_environmental_qda.png")

In [64]:
fig = go.Figure(layout=go.Layout(width=700, height=500
    ))

#title=go.layout.Title(text="Precision and Recall by Threshold")

fig.add_trace(go.Scatter(name="Precision", x=thresholds, y=precision[:-1]))
fig.add_trace(go.Scatter(name="Recall", x=thresholds, y=recall[:-1]))
fig.update_xaxes(title="Threshold")
fig.update_yaxes(title="Proportion")
fig

fig.write_image("images/results/PCR_threshold_environmental_qda.png")

In [65]:
# Generate a range of threshold values
thresholds = np.arange(0.01, 1.00, 0.01)

# Loop through threshold values and calculate F1 score for each
f1_scores = []
for threshold in thresholds:
    y_pred_2 = (y_pred >= threshold).astype(int)
    f1_scores.append(f1_score(y_test, y_pred_2))

# Find optimal threshold value that maximizes F1 score
optimal_threshold = thresholds[np.argmax(f1_scores)]

# Evaluate the model performance
y_pred_2 = (y_pred >= optimal_threshold).astype(int)
print('Accuracy:', accuracy_score(y_test, y_pred_2))
print('Precision:', precision_score(y_test, y_pred_2))
print('Recall:', recall_score(y_test, y_pred_2))
print('F1 score:', f1_score(y_test, y_pred_2))

Accuracy: 0.9794307315988436
Precision: 0.19298245614035087
Recall: 0.19130434782608696
F1 score: 0.19213973799126638


In [66]:
# Print optimal threshold
print('optimal threshold:', optimal_threshold)

optimal threshold: 0.22


In [67]:
# Display confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_2).ravel()
print('tn:', tn, 'fp:', fp, 'fn:', fn, 'tp:', tp)

tn: 8787 fp: 92 fn: 93 tp: 22


### Combine Figures

In [72]:
# Combined ROC Curve
fig = go.Figure()

fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_layout(
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    yaxis=dict(scaleratio=1),
    xaxis=dict(constrain='domain'),
    width=600, height=500,
)
#title = 'ROC Curves for the Environmental Pillar'

fig.add_trace(go.Scatter(x=fpr_lr, y=tpr_lr, name=f"LR (AUC={auc_lr:.2f})", mode='lines'))
fig.add_trace(go.Scatter(x=fpr_lgbm, y=tpr_lgbm, name=f"GB (AUC={auc_lgbm:.2f})", mode='lines'))
fig.add_trace(go.Scatter(x=fpr_qda, y=tpr_qda, name=f"QDA (AUC={auc_qda:.2f})", mode='lines'))

fig.show()

fig.write_image("images/final/environmental/ROC_environmental_combined.png")

pio.write_image(fig, 'images/final/environmental/ROC_environmental_combined.png', scale=5, width=600, height=500)

In [77]:
# Combined PRC Curve
fig = go.Figure()

fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=baseline, y1=baseline
)

fig.update_layout(
    xaxis_title='Recall',
    yaxis_title='Precision',
    yaxis=dict(scaleratio=1),
    xaxis=dict(constrain='domain'),
    width=600, height=500,
)

#title = 'PRC Curves for the Environmental Pillar'

fig.add_annotation(text=f'Baseline: {baseline:.4f}',
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=1.65,
                    y=0.01,
                    bordercolor='black',
                    borderwidth=1,
                    font=dict(size=25,),)

fig.add_trace(go.Scatter(x=recall_lr, y=precision_lr, name=f"LR (AUC={auc_lr:.2f})", mode='lines'))
fig.add_trace(go.Scatter(x=recall_lgbm, y=precision_lgbm, name=f"GB (AUC={auc_lgbm:.2f})", mode='lines'))
fig.add_trace(go.Scatter(x=recall_qda, y=precision_qda, name=f"QDA (AUC={auc_qda:.2f})", mode='lines'))

fig.show()

fig.write_image("images/final/environmental/PRC_environmental_combined.png")

pio.write_image(fig, 'images/final/environmental/PRC_environmental_combined.png', scale=5, width=600, height=500)